# Title
---

Explicar aqui para que es este notebook
Poner la url del dataset original

### Dependencies

In [ ]:
import pandas as pd
import shutil
from random import randint
from numpy import nan

# Reading Data
---
Hablar aqui un poco del dataset y de cuantos datos se van a seleccionar y de que se va a usar pandas.

Poner aqui la explicacion de lo que se va  ahacer en el codigo de abajo.

In [ ]:
def add_reviews_from_to(source, dest, random_choice, *new_cols):
    old_reviews = pd.read_csv(source)
    print(old_reviews)
    try:
        extended_reviews = pd.read_csv(dest)
    except:
        
        old_cols = list(old_reviews.columns)
        cols_names = old_cols[:-1] + list(new_cols) + [old_cols[-1]]
        
        extended_reviews = pd.DataFrame(columns=cols_names)
        
    print(extended_reviews)
    #copiar todo el dataset
    if not random_choice:
        for col in old_reviews:
            extended_reviews[col] = old_reviews[col]
    else:
        count_revs = random_choice
        while count_revs > 0:
            rnd = randint(0, old_reviews.shape[0] - 1)
            review = old_reviews['Review'].iloc[rnd]
            sentiment = old_reviews['Sentiment'].iloc[rnd]
            
            if sentiment == 0:
                sentiment = 'negative'
            if sentiment == 1:
                sentiment = 'positive' 
                        
            if review not in extended_reviews['Review'].values:
                count_revs -= 1
                new_row = pd.DataFrame({'Review': [review], 'Sentiment': [sentiment]})
                extended_reviews = pd.concat([extended_reviews, new_row], ignore_index=True)
    
    extended_reviews.to_csv(dest, index=False)

Explicar aqui lo que se hace debajo

In [ ]:
add_reviews_from_to('data/reviews.csv', 'data/extended_reviews.csv', 0, 'Race')
add_reviews_from_to('data/train_data.csv', 'data/extended_reviews.csv', 80)
